In [1]:
import numpy

In [2]:
import h5py
def load_from_hdf5 ( infile ) :
    import os

    # check if file exists
    if not os.path.isfile( infile ) :
        raise IOError( f"file {infile} does not exist." )
        
    indict = {}
    with h5py.File(infile, 'r') as f :
        indict['metadata'] = dict(f.attrs)
        for name, todict in f.items() :
            try :
                indict[name] = { k : todict.get(k)[:] for k in todict.keys() }
            except Exception as err :
                warnings.warn(
                    f"- Skipping group '{name}' as it raises "
                    f"exception {type(err).__name__} with message:\n  '{err}'"
                )
            kmeta = list(todict.attrs.keys())
            vmeta = list(todict.attrs.values())
            if len(kmeta)>0 and len(vmeta>0) :
                indict['metadata'] = dict(zip(kmeta,vmeta))
        return indict

In [3]:
catalogues = 'catalogue_hi_slice{0:0>2d}.hdf5'

# Statistics: 1-point

## redshift distribution (IM: z<3, Gxys: z<0.5)

Computed as

$$\mathcal{N}(z) = \dfrac{1}{N_\text{tot}}\cdot\dfrac{\text{d}N(z)}{\text{d}z}$$

In [4]:
z_bins = numpy.linspace(0,3,25)

In [5]:
zdist = numpy.zeros(z_bins.size-1, dtype=float)
for ii in range(3) :
    data = load_from_hdf5(catalogues.format(ii))
    red = data['galaxies']['Red']
    zdist += numpy.histogram( red, bins=z_bins )[0]

## Omega_HI

Defined as

$$\Omega_\text{HI}(z) = \dfrac{\rho_\text{HI}(z)}{\rho_\text{crit}(z=0)}\approx \dfrac{\text{d}M_\text{HI}^\text{tot}(z)}{\text{d}V(z)}\cdot\dfrac{1}{\rho_\text{crit}(z=0) }$$

We want 
* the binned total mass 
* volume of the redshift slice (in your cosmology)
* critical density (in your cosmology)

In [6]:
z_bins = numpy.linspace(0,3,7)

In [7]:
zmhi = numpy.zeros(z_bins.size-1, dtype=float)
for ii in range(3) :
    data = load_from_hdf5(catalogues.format(ii))
    
    red = data['galaxies']['Red']
    mhi = data['galaxies']['Mhi']
    dig = numpy.digitize(red, z_bins)-1
    numpy.add.at( zmhi, dig, mhi )

## MHI-Mh relation

We want
* the histograms with the total counts in the two dimensions ($M_\text{halo}$, $M_\text{HI}$) 
* divided in redshift bins?

In [8]:
MDM_bins = numpy.logspace(7,15,1001)
MHI_bins = numpy.logspace(1,12,1001)
z_bins = numpy.linspace(0,3,7)

> **without** redshift binning

In [9]:
# no redshift bins
MHIMh = numpy.zeros((MDM_bins.size-1,MHI_bins.size-1), dtype=float)
for ii in range(3) :
    data = load_from_hdf5(catalogues.format(ii))
    
    # Extract relevant data
    mdm = data['galaxies']['Mdm']   # Halo mass
    mhi = data['galaxies']['Mhi'] # HI mass
    
    # Sum counts
    MHIMh += numpy.histogram2d(mdm, mhi, bins=(MDM_bins, MHI_bins))[0]

> **with** redshift binning

In [10]:
# with redshift bins
MHIMh = numpy.zeros((z_bins.size-1,MDM_bins.size-1,MHI_bins.size-1), dtype=float)
for ii in range(3) :
    data = load_from_hdf5(catalogues.format(ii))    
    
    # Extract relevant data
    red = data['galaxies']['Red']     # Redshift
    mdm = data['galaxies']['Mdm']     # Halo mass
    mhi = data['galaxies']['Mhi'] # HI mass
    
    # Compute bin indices
    z_dig = numpy.digitize(red, z_bins) - 1 
    mdm_dig = numpy.digitize(mdm, MDM_bins) - 1
    mhi_dig = numpy.digitize(mhi, MHI_bins) - 1

    # Filter valid indices to avoid out-of-bound errors
    valid = (
        (0 <= z_dig) & (z_dig < z_bins.size - 1) &
        (0 <= mdm_dig) & (mdm_dig < MDM_bins.size - 1) &
        (0 <= mhi_dig) & (mhi_dig < MHI_bins.size - 1)
    )
    
    # Sum counts
    numpy.add.at(MHIMh, (z_dig[valid], mdm_dig[valid], mhi_dig[valid]), 1)

## HI mass function

We want 
* the mass binned counts 
* volume of the redshift slice (in your cosmology)

In [11]:
MHI_bins = numpy.logspace(1,12,51)
z_bins = numpy.linspace(0,0.5,5)

In [12]:
NMHI = numpy.zeros((z_bins.size-1,MHI_bins.size-1), dtype=float)
for ii in range(3) :
    data = load_from_hdf5(catalogues.format(ii)) 
    
    # Extract relevant data
    red = data['galaxies']['Red']     # Redshift
    mhi = data['galaxies']['Mhi'] # HI mass
    
    # Compute bin indices
    z_dig = numpy.digitize(red, z_bins) - 1 
    mhi_dig = numpy.digitize(mhi, MHI_bins) - 1

    # Filter valid indices to avoid out-of-bound errors
    valid = (
        (0 <= z_dig) & (z_dig < z_bins.size - 1) &
        (0 <= mhi_dig) & (mhi_dig < MHI_bins.size - 1)
    )
    
    # Sum counts
    numpy.add.at(NMHI, (z_dig[valid], mhi_dig[valid]), 1)

# 2-point


## Power spectrum on a box

Paco's python library + tutorial (it's pip-installable):
* [Density field tutorial](https://pylians3.readthedocs.io/en/master/construction.html)
* [Pk tutorial](https://pylians3.readthedocs.io/en/master/Pk.html)

Minimal example usage:
```python
import numpy
import Pk_library as PKL

Npix = 2**8
boxside = 512.0 # cMpc/h
pixsize = boxside/Npix
Npart = 1024**3 # just as an example

# density contrast (uniform for testing)
delta = numpy.random.default_rng().uniform(
    size=(Npix,Npix,Npix)
).astype(numpy.float32)

# allocate power spectrum object (here the computation happens)
Pk = PKL.Pk(
    delta,        # the density contrast
    boxside,      # to get the scale
    axis=0,  
    MAS='NGP',    # different possible mass assignment choices, here NGP=Nearest Grid Point  
    threads=8,    # parallel threads (shared-memory parallel)
    verbose=True 
)

# extract the 3D P(k) and corresponding k grid (no computation here)
kh  = Pk.k3D
Pk0 = Pk.Pk[:,0] #monopole

# limits 
shotnoise = boxside**3/Npart # depends on the density of objects 
kNyq = 2*numpy.pi / pixsize  # Nyquist frequency
kMin = 2*numpy.pi / boxside  # Maximum scale for the given box
```
```
> Computing power spectrum of the field...
> Time to complete loop = 0.70
> Time taken = 0.76 seconds
```

## Cls projected map

Compute at will, provide 
* the $\mathcal{l}$ 
* corresponding $\mathcal{C}(\mathcal{l})$ 
* the $f_\text{sky}$
* the shot noise

## bias?


# Output format

To ease the comparison process, let's stick to the same file format. ``hdf5`` is portable and compact and includes natively metadata.
Whatever the method used to compute the statistics above it shouldn't be too difficult to pack them into a standardized ``hdf5``.

The quantities to store are:
* the redshift distribution ``zdist``
* the density parameter of hydrogen (in terms of total hydrogen mass at varying redshift ``zmhi`` and volume per redshift bin ``Volume``)
* the $M_\text{HI}$ to $M_\text{halo}$ mass ratio (in terms of 2D hystograms with number counts per bin, ``MHIMh``)
* the HI mass function (in terms of number of objects of given mass per bin ``NMHI`` and volume per redshift bin ``Volume``)

> For all of the above it will not be necessary to store the used bins as we are aiming to have everything computed on the same grid.
> We are nevertheless asking for all those quantities that directly depend on your cosmological parameters (e.g. the **comoving volume in h-units** of the redshift bin) 

2 point statistics instead will be re-binned a posteriori, you can give us
* the power spectrum and related quantities:
    - ``kh``
    - ``Pk0``
    - ``shotnoise``
    - ``kNyq``
    - ``kMin``
* the Cells and related quantities:
    - ``Cell``
    - ``ell``
    - ``shot``
    - ``fsky``

In python you would do something like this

```python
import h5py

outfile = '/path/to/measurements/YourName.hdf5'
with h5py.File( outfile, "w" ) as f :
            
    # store eventual meta-data
    #f.attrs[''] = 

    ########################################################
    # store groups:
    
    # Redshift distribution
    group_zdist = f.create_group('redshift_distribution')
    group_zdist.create_dataset( 'zdist', data = zdist )
    
    # Density parameter
    group_omega = f.create_group('omega_hi')
    group_omega.create_dataset( 'zmhi', data = zmhi )
    group_omega.create_dataset( 'Vz', data = vslice )
    
    # MHIMh relation
    group_mhimh = f.create_group('MHIMh')
    group_mhimh.create_dataset('NMHI-NMh', data = MHIMh )
    
    # HI mass function
    group_nmhi = f.create_group('nmhi')
    group_nmhi.create_dataset('NMHI', data = NMHI )
    group_nmhi.create_dataset('Vz', data = vslice)
    
    # power spectrum (on a box)
    group_pk = f.create_group('pk')
    group_pk.create_dataset('kh', data = kh)
    group_pk.create_dataset('Pk0', data = Pk0)
    group_pk.attrs['shot'] = shotnoise
    group_pk.attrs['kMin'] = kMin 
    group_pk.attrs['kNyq'] = kNyq
    
    # power spectrum (on a sphere/lightcone)
    group_pk = f.create_group('cell')
    group_pk.create_dataset('ell', data = ell)
    group_pk.create_dataset('Cell', data = Cell)
    group_pk.attrs['shot'] = shotnoise
    group_pk.attrs['fsky'] = fsky
```

## An example reading routine (for completeness)

The cell below shows how it will be possible to read into a python dictionary all the content of a file formatted as above.

Given that you have stored the dataset in a file ``outfile = '/path/to/dataset.h5'``
```python
indict = {}
with h5py.File(outfile, 'r') as f :
     for name, todict in f.items() :
            try :
                indict[name] = { k : todict.get(k)[:] for k in todict.keys() }
            except Exception as err :
                warnings.warn(
                    f"- Skipping group '{name}' as it raises "
                    f"exception {type(err).__name__} with message:\n  '{err}'"
                )
            kmeta = list(todict.attrs.keys())
            vmeta = list(todict.attrs.values())
            if len(kmeta)>0 and len(vmeta>0) :
                indict['metadata'] = dict(zip(kmeta,vmeta))
```